In [3]:
from keras.models import Model
import numpy as np
import matplotlib.image as mpimg
from consts import *
import glob
from sklearn import svm
from skimage import io
from keras.applications import VGG16
import tensorflow as tf
from keras.layers import Dropout, Flatten, Dense

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

input_shape = (IMG_SIZE, IMG_SIZE, depth)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(512, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(2, activation='softmax', name='predictions')(x)
model = Model(base_model.input, x)

model.load_weights(weights_dir + 'cnn_' + str(CLASS_ID) + '.h5')
train_path = base_path + 'images/cropped_resized/train/'
test_path = base_path + 'images/cropped_resized/test/'


train_vecs = []
train_labels = []
n = 0
nmax = 100000

intermed_model = Model(inputs=model.input,
                   outputs=model.layers[-2].output)
intermed_model.summary()

Num GPUs Available:  1
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 120, 120, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 120, 120, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 120, 120, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 60, 60, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 60, 60, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 60, 60, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 

In [4]:
for filepath in glob.glob(train_path + '*.*'):
    if n > nmax:
        break
    if USE_GREY:
        im = io.imread(filepath, as_gray=USE_GREY)
        im = np.expand_dims(im, axis=2)
    else:
        im = mpimg.imread(filepath)
    im -= 0.5
    im = np.expand_dims(im, axis=0)
    layer_output = intermed_model(im)
    train_vecs.append(layer_output[0])

    if CLASS_NAMES[CLASS_ID] in filepath:
        train_labels.append(1)
    else:
        train_labels.append(0)

    n += 1


train_vecs = np.array(train_vecs)
train_labels = np.array(train_labels)
print(train_vecs.shape)
print(train_labels.shape)

(27070, 512)
(27070,)


In [5]:
clf = svm.SVC(kernel='linear', probability=True)
clf.fit(train_vecs, train_labels)

SVC(kernel='linear', probability=True)

In [6]:
from joblib import dump, load
# svm_weights_path = base_path + 'weights/svm_' + str(CLASS_ID) + '.joblib'

dump(clf, svm_weights_path) 
# clf = load(svm_weights_path) 

['./cuttlefish_data/weights/svm_0.joblib']

In [7]:
test_vecs = []
test_labels = []
n = 0
nmax = 100000
# test_path = base_path + 'images/rcnn_out/'

for filepath in glob.glob(test_path + '*.*'):
    if n > nmax:
        break
    if USE_GREY:
        im = io.imread(filepath, as_gray=USE_GREY)
        im = np.expand_dims(im, axis=2)
    else:
        im = mpimg.imread(filepath)
    
    im -= 0.5
    im = np.expand_dims(im, axis=0)
    
    
    layer_output = np.array(intermed_model(im))
    
    test_vecs.append(layer_output[0])

    if CLASS_NAMES[CLASS_ID] in filepath:
        test_labels.append(1)
    else:
        test_labels.append(0)
    n += 1

In [8]:
svm_preds = clf.predict(test_vecs)
svm_probs = clf.predict_proba(test_vecs)
# print(svm_probs)

wrong_count = 0
for svm_prob, true_label in zip(svm_probs, test_labels):
    if svm_prob[1] > 0.5:
        svm_guess = 1
    else:
        svm_guess = 0
    if svm_guess != true_label:
        wrong_count += 1
print(str(wrong_count) + '/' + str(len(test_labels)))

39/3389
